### Assignment 1 Part III - KG searching, weighting, and embedding

**Overview:** In this notebook, you will learn how to a couple of different ways search to through a large knowledge graph, how to reweight the knowledge graph to favor specific paths, and how to embed nodes in the knowledge graph to try and fill in gaps.   

#### Part I - a couple of different ways search to through a large knowledge graph

These first cells simply set up some variables that will be used below

TODO: get username from the Jupyter and put that in the path

In [45]:
import sys
sys.path.append('/ihome/rboyce/rdb20/.conda/envs/bionf2071/lib/python3.7/site-packages/')

In [46]:
# There can be many paths of the same length so you can adjust how many you want returned
maxNumPaths = 5

You can populate the list in the next cell with the start and end nodes that you will use to search for paths in the knowledge graph. The [PheKnowLator overview](https://user-images.githubusercontent.com/8030363/103158881-11813b00-4780-11eb-8b45-5063765e7645.png) shows all of the ontologies loaded into the knowledge graph as orange boxes. You can replace the start and end nodes with entities that interest you in those ontologies by going to [Ontobee](http://www.ontobee.org/) and obtaining the URIs and then replacing the start and end node pairs in the s_o_tpl_L list. Be sure to set FIRST_TO_PROCESS to the first pair. If you have to restart the knowledge graph searches shown later, you can change that variable.

In [3]:
s_o_tpl_L = [
    ("http://purl.obolibrary.org/obo/HP_0000716","http://purl.obolibrary.org/obo/DOID_680"),
    ("http://purl.obolibrary.org/obo/HP_0000716","http://purl.obolibrary.org/obo/HP_0001297"),
    ("http://purl.obolibrary.org/obo/HP_0000716","http://purl.obolibrary.org/obo/HP_0001635"),
    ("http://purl.obolibrary.org/obo/HP_0000716","http://purl.obolibrary.org/obo/HP_0002383"),
    ("http://purl.obolibrary.org/obo/HP_0000716","http://purl.obolibrary.org/obo/HP_0002621")
]
## Leave as None unless you have to restart this cell before it has completely processed all tuples.
## Otherwise, replace this tuple with the first tuple to process completely by the program 
FIRST_TO_PROCESS =  ("http://purl.obolibrary.org/obo/HP_0000716","http://purl.obolibrary.org/obo/DOID_680")

The libraries needed to import the graph and work with it

In [4]:
import os
import os.path
import networkx as nx
import json
import urllib
import traceback
import sys
from itertools import islice
from rdflib import Graph, URIRef, BNode, Namespace, Literal
from rdflib.namespace import RDF, OWL
import smart_open
from node2vec import Node2Vec

The next two cell are the graph loading step. The graph is the output of the PheKnowLator process including filtering out OWL semantics. The sources used in this version of the knowledge graph are listed on [this Wiki page](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources). The graph takes more than 15 gigs of RAM once loaded.

In [5]:
GRAPHPATH = "PheKnowLator_full_InverseRelations_NotClosed_OWLNETS_Networkx_MultiDiGraph_REWEIGHT_NO_DISJOINT.gpickle"

In [6]:
# Reloading the graph produced from the PheKnowLator workflow
# The graph 
nx_mdg = nx.read_gpickle(GRAPHPATH)

The next cell defines several functions that will help to explain the results of path searches.

In [7]:
ontobee_service = "http://sparql.hegroup.org/sparql/"


def query(q,epr,f='application/sparql-results+json'):
    """Function that uses urllib/urllib2 to issue a SPARQL query.
       By default it requests json as data format for the SPARQL resultset"""

    try:
        params = {'query': q}
        params = urllib.parse.urlencode(params)
        opener = urllib.request.build_opener(urllib.request.HTTPHandler)
        request = urllib.request.Request(epr+'?'+params)
        request.add_header('Accept', f)
        request.get_method = lambda: 'GET'
        url = opener.open(request)
        return url.read()
    except Exception as e:
        traceback.print_exc(file=sys.stdout)
        raise e


def pathQuery(pth):
    """Given a single path (list of rdflib objects), run a sparql query to retrieve descriptive information 
       that will help construct a narrative explanation"""
    
    uriL = [x.toPython() for x in filter(lambda x: type(x) == URIRef, pth)]
    subjectStr = ''
    objectStr = ''
    for uri in uriL:
        subjectStr = subjectStr + '?s = <' + uri + '> || '
        objectStr = objectStr + '?o = <' + uri + '> || '
    subjectStr = subjectStr[:-3] # ?s = <..> || ?s = <...>... the URIs for the subject/object entities in the path
    objectStr = objectStr[:-3] # ?o = <..> || ?o = <...>... the URIs for the subject/object entities in the path
   
    q = '''
  prefix obo:<http://purl.obolibrary.org/obo/>
  prefix owl:<http://www.w3.org/2002/07/owl#>
 
  select distinct ?s ?p ?o ?p_lab ?s_lab_eg ?o_lab_eg
  from <http://phenowlator_merged.owl>
  from <ro_with_imports_AD_mods>
  where {{
    ?s ?p ?o.FILTER(({}) && ({})) 
    OPTIONAL{{
     ?p rdfs:label ?p_lab.
   }}
   OPTIONAL{{
     ?egM_s <http://dikb.org/ad#obo_mapping> ?s.
     ?egM_s rdfs:label ?s_lab_eg.
   }}
   OPTIONAL{{
     ?egM_o <http://dikb.org/ad#obo_mapping> ?o.
     ?egM_o rdfs:label ?o_lab_eg.
   }}
  }}
'''.format(subjectStr,objectStr)
    
    return q


def missingLabelQuery(uri, endpoint):
    query_string = '''
SELECT distinct ?lab
WHERE {{ 
 <{}> rdfs:label ?lab. 
}}
'''.format(uri)
    json_string = query(query_string, endpoint)
    resultset = json.loads(json_string)
    rsltsD = {}
    for b in resultset["results"]["bindings"]:
        if not b.get('lab'):
            return None
        
        label = b['lab']['value']
        if label:
            return label
        else:
            return None

def constructPathNarData(pth, sparql_service, debug=False):
    """ Iterate through the path to organize a narrative
        in: pth - a list of rdflib URIRef and BNode objects
        in: sparql_service - URL to the sparql endpoint
    """
    query_string = pathQuery(pth)
    if debug:
        print('[DEBUG] Query:\n' + query_string)
        
    json_string = query(query_string, pheknowlator_service)
    resultset = json.loads(json_string)
    print("[INFO] Number of results: " + str(len(resultset["results"]["bindings"])))
    
    # Re-organize the results set to be a dict keyed by the subject uri
    rsltsD = {}
    for b in resultset["results"]["bindings"]:
        s = b['s']['value']
        if rsltsD.get(s):
            rsltsD[s].append(b)
        else:
            rsltsD[s] = [b]
    
    narrative = ""     
    for i in range(0,len(pth)):
        if i == len(pth) - 1:
            break
        
        n1 = pth[i]
        n2 = pth[i+1]
    
        if not (type(n1) == URIRef and type(n2) == URIRef):
            narrative += "----- BLANK NODE STEP ----"
            continue
        else:
            # locate the results dict that relates n1 to n2 in a subject, predicate, object triple
            o_d = None
            for d in rsltsD.get(n1.toPython()):
                if d['o']['value'] == n2.toPython():
                    o_d = d
                    break
        
            if not o_d:
                print("ERROR: Unable to find a triple relating {} to {} in path {}".format(n1.toPython(),n2.toPython(),pth))
            else:
                o_lab = '<no label found>'
                o_node_id = o_d['o']['value'].split('/')[-1]
                if nodeLabD.get(o_node_id):
                    o_lab = nodeLabD[o_node_id]
                else:
                    if label_cache.get(o_d['o']['value']):
                        o_lab = label_cache[o_d['o']['value']]
                    else:
                        ql = missingLabelQuery(o_d['o']['value'],ontobee_service)
                        if ql:
                            label_cache[o_d['o']['value']] = ql
                            o_lab = ql
                    
                    if o_lab == '<no label found>' and o_d.get('o_lab_eg'):
                        o_lab = o_d['o_lab_eg']['value'] + ' (UMLS label)'

                s_lab = '<no label found>'
                s_node_id = o_d['s']['value'].split('/')[-1]
                if nodeLabD.get(s_node_id):
                    s_lab = nodeLabD[s_node_id]
                else:
                    if label_cache.get(o_d['s']['value']):
                        s_lab = label_cache[o_d['s']['value']]
                    else:
                        ql = missingLabelQuery(o_d['s']['value'],ontobee_service)
                        if ql:
                            label_cache[o_d['s']['value']] = ql
                            s_lab = ql  
                    
                    if s_lab == '<no label found>' and o_d.get('s_lab_eg'):
                        s_lab = o_d['s_lab_eg']['value'] + ' (UMLS label)'

                if o_d.get('p_lab'):                      
                    narrative += '{}\t{}\t{}\t{}\t{}\t{}\n'.format(
                            s_lab,o_d['p_lab']['value'],o_lab,
                            o_d['s']['value'],o_d['p']['value'],o_d['o']['value']
                           )                          
                else:
                    narrative += '{}\t{}\t{}\t{}\t{}\t{}\n'.format(
                            s_lab,o_d['p']['value'].replace('http://www.w3.org/2000/01/rdf-schema#',''),o_lab,
                            o_d['s']['value'],o_d['p']['value'],o_d['o']['value']
                           )                                          
    return narrative

The next cell loads a file that has all of the node labels which were stripped from the graph when OWL semantics were removed.

In [8]:
nodeLabD = {}
f = open('PheKnowLator_full_InverseRelations_NotClosed_NoOWLSemantics_NodeLabels.txt','r')
buf = f.read()
f.close()
nodLabL = buf.split('\n')
for line in nodLabL:
    spL = line.split('\t')
    if len(spL) > 1:
        nodeLabD[spL[0]] = spL[1]

This function conducts a shortest path search given a graph source and target k results are returned

In [9]:
def k_shortest_paths(G, source, target, k, weight='weight'):
    return list(islice(nx.all_shortest_paths(G, source, target, weight=weight), k))

This function conducts the simple path searches given a graph source and target and an integers with the shortest path length known 
The shortest path lengths is used so that already seen simple paths are not returned
k results are returned

In [10]:
def k_simple_paths(G, source, target, k, shortestLen):
    paths = nx.all_simple_paths(G, source, target, cutoff=shortestLen+20)
    path_l = []
    i = 0
    while i < k:
        try:
            print('[info] applying next operator to search for a simple path of max length {}'.format(shortestLen+20))
            path = next(paths)
        except StopIteration:
            break
        print('[info] Simple path found of length {}'.format(len(path))) 
        if len(path) > shortestLen:
            print('[info] Simple path length greater than shortest path length ({}) so adding to results'.format(shortestLen))
            path_l.append(path)
        i += 1
    return path_l      

In [11]:
# This is the endpoint where an RDF version of the PheKnowLator graph resides
pheknowlator_service = "https://dbmi-icode-01.dbmi.pitt.edu/sparql"

In [ ]:
# There can be allot of repetition when doing certain search patterns (e.g., confounders) so set up a cache 
processed_tpl_cache = []
# queried label cache
label_cache = {}

The next cell conducts the shortest path searches

In [ ]:
shortestPathsLens = [] # we track the shortest paths from source to target for when we do simple path searches
for tpl in s_o_tpl_L:      
    if FIRST_TO_PROCESS:
        if str(tpl) != str(FIRST_TO_PROCESS):
            print('INFO: skipping tuple because it is not FIRST_TO_PROCESS:' + str(tpl))
            continue
        else:
            FIRST_TO_PROCESS = None
    
    (s,o) = tpl    
    startNd = URIRef(s) 
    endNd = URIRef(o) 
       
    print('INFO: Processing {} and {}:\n'.format(s,o))
                
    try:
        pthL = k_shortest_paths(nx_mdg,startNd,endNd,maxNumPaths)
    except nx.NetworkXNoPath:
        print('INFO: No results in the path search.\n')
        continue
    except nx.NodeNotFound:
        print('INFO: The source node does not exist in the Knowledge Graph.\n')
        continue

    narL = []
    c = -1
    for path in pthL:
        if c == -1:
            shortestPathsLens.append(len(path))
            
        c += 1
        nar = constructPathNarData(path,pheknowlator_service,debug=False)
        print('\n\nINFO: SHORTEST PATH {}:\n'.format(c))
        print(nar)
        print('\n\n')

The next cell conducts the simple path searches but in a way to ignores the shortest paths

In [ ]:
maxNumPaths = 3  # we search for fewer paths b/c this can take much longer than the shortest path searches  
tplCnt = 0
for tpl in s_o_tpl_L:      
    if FIRST_TO_PROCESS:
        if str(tpl) != str(FIRST_TO_PROCESS):
            print('INFO: skipping tuple because it is not FIRST_TO_PROCESS:' + str(tpl))
            continue
        else:
            FIRST_TO_PROCESS = None
    
    (s,o) = tpl    
    startNd = URIRef(s) 
    endNd = URIRef(o) 
       
    print('INFO: Processing {} and {}:\n'.format(s,o))
                
    try:
        pthL = k_simple_paths(nx_mdg, startNd, endNd, maxNumPaths, shortestPathsLens[tplCnt])
        tplCnt += 1
    except nx.NetworkXNoPath:
        print('INFO: No results in the path search.\n')
        continue
    except nx.NodeNotFound:
        print('INFO: The source node does not exist in the Knowledge Graph.\n')
        continue

    narL = []
    c = -1
    for path in pthL:
        c += 1
        nar = constructPathNarData(path,pheknowlator_service,debug=False)
        print('\n\nINFO: SIMPLE PATH {}:\n'.format(c))
        print(nar)
        print('\n\n')

**E7** - Questions about path searches

1. In part II of Assignment I you saw some SPARQL queries over a small knowledge graph created from predications extracted by machine reading with SemMedDB. In this part of the Assignment I you are working with a much larger and more complex graph constructed using ontologies and several large data sources (go back and review sources used in this version of the knowledge graph are listed on [this Wiki page](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources)). We could SPARQL for this graph but we are using graph search instead. Make some obervations in your own words about some differences between the two approaches. 

2. Take a close look at the results of the shortest path searches and make some comments about how potentially factual and useful the path narratives would be for providing a mechanistic explanation of the relationship between the start and end nodes. 

3. Similar to the previous question, take a close look at the results of the SIMPLE path searches and make some comments about how potentially factual and useful the path narratives would be for providing a mechanistic explanation of the relationship between the start and end nodes. Also, what differences between the simple and shortes path searches stand out to you.

4. Do you have any questions for me about what you see?

#### Part II - how to reweight the knowledge graph to favor specific paths

There are a number of graph metrics that could be use to reweight edges in the graph (see https://networkx.org/documentation/stable/reference/algorithms/centrality.html) Here, we pick a simple measure to calculate called degree centrality.

In [ ]:
## obtain a dictionary of node degree centrality for all nodes using the default parameters
centrality = nx.degree_centrality(nx_mdg)

In [ ]:
# Using Alzheimer's disease as an example 
centrality[URIRef('http://purl.obolibrary.org/obo/HP_0002511')]

In [ ]:
# Using Cell as an example
centrality[URIRef('http://purl.obolibrary.org/obo/CL_0000000')]

In [ ]:
# We are going to use node degree centrality when reweighting the graph so we need to calculate the measure for all nodes
f = open('centrality.tsv','w')
f.write('node\tdegree_centrality\n')
for k,v in centrality.items():
    f.write('{}\t{}\n'.format(k.toPython(),v))
f.close()

The next cell creates a new graph  copying and reweighting the current graph using degree centrality

In [ ]:
nx_mdg_cntr = nx.MultiDiGraph()

# the reweight graph will remove 'disjoint' and 'domain' predicates  and apply a 
edge_key = -1 # an int that we will increment to uniquely identify edges
for s, o, data in nx_mdg.edges(data=True):
    p = data['predicate']
    
    edge_key += 1
    
    ## default weight = 2
    weight = 2.0
    
    # Skip disjoint and domain predicates
    if p.toPython() == 'http://www.w3.org/2002/07/owl#disjointWith' or p.toPython() == 'http://www.w3.org/2000/01/rdf-schema#domain':
        continue    
           
    # Add to the weight the degree centrality of each node
    if centrality.get(p):
        weight = weight + centrality[p]
    if centrality.get(o):
        weight = weight + centrality[o]
    
    # add the edge to the graph giving it a unique key and weight
    nx_mdg_cntr.add_edge(s, o, **{'predicate': p,'key': edge_key, 'weight':weight})

In [ ]:
nx.write_gpickle(nx_mdg_cntr,'PheKnowLator_full_InverseRelations_NotClosed_OWLNETS_DEGREE_CNTRLTY_REWEIGHT_NO_DISJOINT.gpickle')

Conduct shortest and simple path searches with the reweighted graph

In [ ]:
maxNumPaths = 3 
shortestPathsLens = [] # we track the shortest paths from source to target for when we do simple path searches
for tpl in s_o_tpl_L:      
    if FIRST_TO_PROCESS:
        if str(tpl) != str(FIRST_TO_PROCESS):
            print('INFO: skipping tuple because it is not FIRST_TO_PROCESS:' + str(tpl))
            continue
        else:
            FIRST_TO_PROCESS = None
    
    (s,o) = tpl    
    startNd = URIRef(s) 
    endNd = URIRef(o) 
       
    print('INFO: Processing {} and {}:\n'.format(s,o))
                
    try:
        pthL = k_shortest_paths(nx_mdg_cntr,startNd,endNd,maxNumPaths)
    except nx.NetworkXNoPath:
        print('INFO: No results in the path search.\n')
        continue
    except nx.NodeNotFound:
        print('INFO: The source node does not exist in the Knowledge Graph.\n')
        continue

    narL = []
    c = -1
    for path in pthL:
        if c == -1:
            shortestPathsLens.append(len(path))
            
        c += 1
        nar = constructPathNarData(path,pheknowlator_service,debug=False)
        print('\n\nINFO: SHORTEST PATH {}:\n'.format(c))
        print(nar)
        print('\n\n')

In [ ]:
# Conduct SIMPLE path searches with the reweighted graph
maxNumPaths = 3  
tplCnt = 0
for tpl in s_o_tpl_L:      
    if FIRST_TO_PROCESS:
        if str(tpl) != str(FIRST_TO_PROCESS):
            print('INFO: skipping tuple because it is not FIRST_TO_PROCESS:' + str(tpl))
            continue
        else:
            FIRST_TO_PROCESS = None
    
    (s,o) = tpl    
    startNd = URIRef(s) 
    endNd = URIRef(o) 
       
    print('INFO: Processing {} and {}:\n'.format(s,o))
                
    try:
        pthL = k_simple_paths(nx_mdg_cntr, startNd, endNd, maxNumPaths, shortestPathsLens[tplCnt])
        tplCnt += 1
    except nx.NetworkXNoPath:
        print('INFO: No results in the path search.\n')
        continue
    except nx.NodeNotFound:
        print('INFO: The source node does not exist in the Knowledge Graph.\n')
        continue

    narL = []
    c = -1
    for path in pthL:
        c += 1
        nar = constructPathNarData(path,pheknowlator_service,debug=False)
        print('\n\nINFO: SIMPLE PATH {}:\n'.format(c))
        print(nar)
        print('\n\n')

**E8** - questions about graph editing and reweighting

1. Think about how degree centrality is being used here to reweight edges in the graph. If the goal is to obtain mechanistically accurate narratives, does using this measure to reweight make sense?

2. Imagine that the goal is to avoid the inclusion several 'hub' nodes in path results. Look over the [centrality measures](https://networkx.org/documentation/stable/reference/algorithms/centrality.html) and justify if another which measure(s) might be better than degree centraity to accomplish this task.

3. How could you use knowledge about the Relation Ontology to bias path search results to favor certain kinds of nodes (e.g., pathways) over others (e.g., genes) (hint: look at the ontologies and RO entities [here](https://user-images.githubusercontent.com/8030363/103158881-11813b00-4780-11eb-8b45-5063765e7645.png))?



### Part III - how to embed nodes in the knowledge graph to try and fill in gaps

We're going to tackle link prediction as a supervised learning problem on top of node representations/embeddings. The embeddings are computed with the unsupervised node2vec algorithm. After obtaining embeddings, a binary classifier can be used to predict a link, or not, between any two nodes in the graph. Various hyperparameters could be relevant in obtaining the best link classifier -

There are four steps:

1. Obtain embeddings for each node
2. For each set of hyperparameters, train a classifier
3. Select the classifier that performs the best
4. Evaluate the selected classifier on unseen data to validate its ability to generalise

This part of the notebook has been adapted from a demo of the StellarGraph library (https://stellargraph.readthedocs.io/en/stable/index.html) for link prediction using the node2vec algorithm.

<a name="refs"></a>
**References:** 

[1] Node2Vec: Scalable Feature Learning for Networks. A. Grover, J. Leskovec. ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), 2016.

In [53]:
### First, we derive a simple undirected graph that has only the RO interacts_with and molecularly_interacts_with as predicates (edges)
## http://purl.obolibrary.org/obo/RO_0002434 - interacts with
## http://purl.obolibrary.org/obo/RO_0002436 - molecularly interacts with
##
## Note: We limit the graph size to 50,000 nodes (about 1/5th of the interactin graph size) so that embedding 
## can happen in a relatively short amount of time
##
nx_mdg_interacts = nx.Graph()

# the reweight graph will remove 'disjoint' and 'domain' predicates  and apply a 
maxNodes = 25000
nodeCnt = -1
edge_key = -1 # an int that we will increment to uniquely identify edges
s_count = 0 # some entitites have a huge number of interactions so we will only sample a max of 100 for any entity
s_current = None
for s, o, data in nx_mdg.edges(data=True):
    if s == s_current:
        s_count += 1
        if s_count > 100:
            continue
    else:
        s_current = s
        s_count = 0 
        
    p = data['predicate']
    
    edge_key += 1
    
    if not (p.toPython() == 'http://purl.obolibrary.org/obo/RO_0002434' or p.toPython() == 'http://purl.obolibrary.org/obo/RO_0002436'):
        continue    
         
    # add the edge to the graph giving it a unique key 
    nx_mdg_interacts.add_edge(s, o, **{'predicate': p,'key': edge_key})
    nodeCnt += 1
    if nodeCnt == maxNodes:
        break

In [54]:
nx.write_gpickle(nx_mdg_interacts,'PheKnowLator_full_InverseRelations_NotClosed_OWLNETS_INTERACTS.gpickle')

In [12]:
## Uncomment this to reload the 'interacts only' graph from file if needed
# nx_mdg_interacts = nx.read_gpickle('PheKnowLator_full_InverseRelations_NotClosed_OWLNETS_INTERACTS.gpickle')

In [55]:
print(nx.info(nx_mdg_interacts))

Name: 
Type: Graph
Number of nodes: 14521
Number of edges: 24918
Average degree:   3.4320


In [56]:
i = 0 
for d in nx_mdg_interacts.degree():
    if d[1] > 2:        
        print(d)
        i += 1
        if i == 100:
            break

(rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_46024'), 138)
(rdflib.term.URIRef('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000607195'), 7)
(rdflib.term.URIRef('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000557107'), 5)
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_132082'), 115)
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_30563'), 50)
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_84069'), 11)
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_39867'), 209)
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_28420'), 110)
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_27899'), 167)
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/GO_0019825'), 77)
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_3766'), 111)
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_5831'), 3)
(rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_18358'), 3)
(rdflib.t

In [59]:
# A sample of what this new graph looks like
i = 0
for s, o, data in nx_mdg_interacts.edges(data=True):
    if i == 200:
        break
    
    print('{}\t{}\t{}'.format(s,o,data))
    i += 1

http://purl.obolibrary.org/obo/CHEBI_46024	https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000607195	{'predicate': rdflib.term.URIRef('http://purl.obolibrary.org/obo/RO_0002434'), 'key': 142}
http://purl.obolibrary.org/obo/CHEBI_46024	https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000557107	{'predicate': rdflib.term.URIRef('http://purl.obolibrary.org/obo/RO_0002434'), 'key': 5230}
http://purl.obolibrary.org/obo/CHEBI_46024	https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000541206	{'predicate': rdflib.term.URIRef('http://purl.obolibrary.org/obo/RO_0002434'), 'key': 2}
http://purl.obolibrary.org/obo/CHEBI_46024	https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000415806	{'predicate': rdflib.term.URIRef('http://purl.obolibrary.org/obo/RO_0002434'), 'key': 3}
http://purl.obolibrary.org/obo/CHEBI_46024	https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000528786	{'predicate': rdflib.term.URIRef('http:

In [ ]:
# From https://stackoverflow.com/questions/37183749/how-to-find-two-randoms-nodes-with-no-edges-between-them-in-graph
def select_2_random_unconnected_nodes(node_list, graph):
    selected_node = random.choice(list(node_list))
    print('Randomly selected node: {}'.format(selected_node))
  
    # obtain all the nodes connected to the selected node
    connected_nodes = [n for _, n in graph.edges(selected_node)]

    print('All nodes connected to the randomly selected node: {}'.format(connected_nodes + [selected_node]))

    # a feasible node is one not in connected_nodes and also not the first selected_node
    feasible_nodes = [feasible_n for feasible_n in node_list if feasible_n not in connected_nodes + [selected_node]]

    # select a second node from the feasible_nodes list
    select_second_node = random.choice(feasible_nodes)

    return selected_node, select_second_node
  

In [ ]:
import random
nodes = nx_mdg_interacts.nodes()

(selected_node, select_second_node) = select_2_random_unconnected_nodes(nodes,nx_mdg_interacts)
print('\nselected_node: {}\n\nnon-connected node:{}'.format(selected_node, select_second_node))

In [60]:
node2vec = Node2Vec(nx_mdg_interacts, dimensions=30, walk_length=10, num_walks=5, workers=1)

Generating walks (CPU: 1): 100%|██████████| 5/5 [00:02<00:00,  2.09it/s]


In [61]:
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [62]:
model.save('PheKnowLator_full_InverseRelations_NotClosed_OWLNETS_INTERACTS.model')

In [16]:
## Uncomment this and run it to reload the model from a file if you have already created it
from gensim.models import Word2Vec
model = Word2Vec.load('PheKnowLator_full_InverseRelations_NotClosed_OWLNETS_INTERACTS.model')

In [ ]:
print(model.wv.vocab)

In [64]:
# entinostat
model.wv.get_vector('http://purl.obolibrary.org/obo/CHEBI_132082')

array([ 0.5383423 ,  0.13687564, -0.08844738, -0.4704808 ,  0.70583504,
       -0.08056314,  0.3609822 ,  0.7107623 ,  0.3812658 , -0.20181724,
        1.1912521 ,  0.9447325 , -0.9787167 , -0.12856221,  0.876893  ,
       -0.671307  , -0.36094308,  0.2095743 ,  0.03721024, -0.10984633,
        0.5896993 ,  0.4692861 ,  0.24264102,  0.9894832 , -0.7243074 ,
       -0.07927914, -0.05194373, -1.1407896 ,  0.92476183, -0.2196852 ],
      dtype=float32)

In [66]:
model.wv.most_similar('http://purl.obolibrary.org/obo/CHEBI_132082')

[('http://purl.obolibrary.org/obo/CHEBI_9908', 0.9994221925735474),
 ('http://purl.obolibrary.org/obo/PR_000008321', 0.999323308467865),
 ('http://purl.obolibrary.org/obo/PR_P09341', 0.999305248260498),
 ('http://purl.obolibrary.org/obo/PR_P46934', 0.999260425567627),
 ('http://purl.obolibrary.org/obo/CHEBI_37659', 0.9992169737815857),
 ('http://purl.obolibrary.org/obo/CHEBI_16118', 0.9992163181304932),
 ('http://purl.obolibrary.org/obo/CHEBI_6980', 0.9992071986198425),
 ('http://purl.obolibrary.org/obo/PR_000014076', 0.999179482460022),
 ('http://purl.obolibrary.org/obo/CHEBI_28445', 0.9991783499717712),
 ('http://purl.obolibrary.org/obo/PR_Q9HBM1', 0.9991737604141235)]

In [67]:
# positive regulation of nervous system development
model.wv.get_vector('http://purl.obolibrary.org/obo/GO_0051962')

array([ 1.11695072e-02,  2.70528905e-03, -7.11266650e-03,  6.21647900e-03,
        2.50877012e-02, -5.23969159e-03,  9.80134774e-03,  2.69477107e-02,
       -1.24853160e-02, -1.41592976e-03,  1.45947505e-02,  1.83813870e-02,
        5.69508411e-05,  1.03864889e-03,  1.46188363e-02, -1.01599386e-02,
       -1.07991276e-02,  6.58706808e-03, -5.68754971e-03,  5.23295300e-03,
        8.83659907e-03, -1.34815963e-03,  1.75206829e-02,  1.67271942e-02,
       -1.01268003e-02, -1.88999046e-02, -6.08671131e-03, -1.34553462e-02,
       -8.26150994e-04,  6.22922368e-03], dtype=float32)

In [68]:
model.wv.most_similar('http://purl.obolibrary.org/obo/GO_0051962')

[('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000366131',
  0.8030844330787659),
 ('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000619420',
  0.8003867864608765),
 ('http://purl.obolibrary.org/obo/PR_000004339', 0.7990934252738953),
 ('http://purl.obolibrary.org/obo/CHEBI_73237', 0.7943668365478516),
 ('http://purl.obolibrary.org/obo/PR_000005385', 0.7844266891479492),
 ('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000490763',
  0.7814849615097046),
 ('http://purl.obolibrary.org/obo/PR_Q12913', 0.7788965702056885),
 ('http://purl.obolibrary.org/obo/PR_P62837', 0.7776200175285339),
 ('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000672591',
  0.7769244909286499),
 ('http://purl.obolibrary.org/obo/CHEBI_73162', 0.7742947340011597)]

In [69]:
# positive=purine ribonucleoside binding and negative=positive regulation of nervous system development
model.wv.most_similar(positive=['http://purl.obolibrary.org/obo/GO_0032550'],negative=['http://purl.obolibrary.org/obo/GO_0051962'])

[('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000368112',
  0.5666130781173706),
 ('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000470382',
  0.5283974409103394),
 ('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000483991',
  0.5206739902496338),
 ('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000514756',
  0.509303092956543),
 ('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000427021',
  0.5004165172576904),
 ('http://purl.obolibrary.org/obo/GO_1904724', 0.4951947331428528),
 ('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000222597',
  0.468858927488327),
 ('http://purl.obolibrary.org/obo/PR_000002207', 0.4671010971069336),
 ('https://www.ncbi.nlm.nih.gov/gene/90665', 0.4600873589515686),
 ('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000443943',
  0.45980921387672424)]

The next cells train a model for link prediction using the embeddings. We start by building an adjacency matrix and then traversing it to find the positions of the zeros which represent node pairs that are not connected in the graph.  

In [70]:
nodelist = []
nds = nx_mdg_interacts.nodes()
for n in nds:
    nodelist.append(n)

In [71]:
# build adjacency matrix
adj_G = nx.to_numpy_matrix(nx_mdg_interacts,nodelist)

In [72]:
adj_G.shape

(14521, 14521)

In [ ]:
# get unconnected node-pairs
all_unconnected_pairs = [None]*(adj_G.shape[0]*adj_G.shape[1])
all_connected_pairs = [None]*(adj_G.shape[0]*adj_G.shape[1])

# traverse adjacency matrix (iterate columns by rows)
offset = 0
for i in range(adj_G.shape[0]):
  for j in range(offset,adj_G.shape[1]):
    if i != j:
        if adj_G[i,j] == 0:
          all_unconnected_pairs[i+j] = (nodelist[i],nodelist[j])
        else:            
          all_connected_pairs[i+j] = (nodelist[i],nodelist[j])

  offset = offset + 1

In [85]:
import pandas as pd

all_connected_pairs_clean = [x for x in filter(lambda x: x != None, all_connected_pairs)]
node_1_linked = [i[0] for i in all_connected_pairs_clean]
node_2_linked = [i[1] for i in all_connected_pairs_clean]
original_g_df = pd.DataFrame({'node_1':node_1_linked, 
                              'node_2':node_2_linked})

all_unconnected_pairs_clean = [x for x in filter(lambda x: x != None, all_unconnected_pairs)]
node_1_unlinked = [i[0] for i in all_unconnected_pairs_clean]
node_2_unlinked = [i[1] for i in all_unconnected_pairs_clean]
data = pd.DataFrame({'node_1':node_1_unlinked, 
                     'node_2':node_2_unlinked})

# add target variable 'link'
data['link'] = 0

In [86]:
original_g_df.shape

(15524, 2)

In [87]:
len(all_unconnected_pairs_clean)

29037

In [77]:
data.shape

(29037, 3)

In [78]:
import pickle
f = open('original_g_df.pickle','wb')
pickle.dump(original_g_df,f)
f.close()

f = open('data.pickle','wb')
pickle.dump(data,f)
f.close()

In [79]:
original_g_df.head(30)

,node_1,node_2
0,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
1,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
2,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
3,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
4,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
5,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
6,http://purl.obolibrary.org/obo/CHEBI_46024,http://purl.obolibrary.org/obo/GO_0005741
7,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
8,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
9,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...


In [80]:
data.head(30)

,node_1,node_2,link
0,https://uswest.ensembl.org/Homo_sapiens/Transc...,https://uswest.ensembl.org/Homo_sapiens/Transc...,0
1,https://uswest.ensembl.org/Homo_sapiens/Transc...,https://uswest.ensembl.org/Homo_sapiens/Transc...,0
2,https://uswest.ensembl.org/Homo_sapiens/Transc...,https://uswest.ensembl.org/Homo_sapiens/Transc...,0
3,https://uswest.ensembl.org/Homo_sapiens/Transc...,https://uswest.ensembl.org/Homo_sapiens/Transc...,0
4,https://uswest.ensembl.org/Homo_sapiens/Transc...,https://uswest.ensembl.org/Homo_sapiens/Transc...,0
5,https://uswest.ensembl.org/Homo_sapiens/Transc...,https://uswest.ensembl.org/Homo_sapiens/Transc...,0
6,https://uswest.ensembl.org/Homo_sapiens/Transc...,https://uswest.ensembl.org/Homo_sapiens/Transc...,0
7,https://uswest.ensembl.org/Homo_sapiens/Transc...,https://uswest.ensembl.org/Homo_sapiens/Transc...,0
8,https://uswest.ensembl.org/Homo_sapiens/Transc...,https://uswest.ensembl.org/Homo_sapiens/Transc...,0
9,https://uswest.ensembl.org/Homo_sapiens/Transc...,http://purl.obolibrary.org/obo/GO_0005741,0


In [37]:
original_g_df_temp = original_g_df.copy()

In [38]:
original_g_df_temp.head(20)

,node_1,node_2
0,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
1,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
2,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
3,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
4,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
5,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
6,http://purl.obolibrary.org/obo/CHEBI_46024,http://purl.obolibrary.org/obo/GO_0005741
7,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
8,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...
9,http://purl.obolibrary.org/obo/CHEBI_46024,https://uswest.ensembl.org/Homo_sapiens/Transc...


In [88]:
G_orig = nx.from_pandas_edgelist(original_g_df, "node_1", "node_2", create_using=nx.Graph())
initial_node_count = nx.number_of_nodes(G_orig)
print('initial_node_count: {}'.format(initial_node_count))

initial_node_count: 10853


In [89]:
original_g_df_temp = original_g_df.copy()
G_temp = nx.from_pandas_edgelist(original_g_df_temp.drop(2), "node_1", "node_2", create_using=nx.Graph())
temp_node_count = nx.number_of_nodes(G_temp)
print('temp_node_count: {}'.format(temp_node_count))

temp_node_count: 10852


In [90]:
len(original_g_df.index.values)

15524

In [91]:
original_g_df_temp = original_g_df.copy()

# empty list to store removable links
omissible_links_index = []

ctr = 0
ctr2 = -1
for i in original_g_df.index.values:
    ctr2 += 1
    #if ctr2 == 2000:
    #    break
    
    # remove a node pair and build a new graph
    G_temp = nx.from_pandas_edgelist(original_g_df_temp.drop(i), "node_1", "node_2", create_using=nx.Graph())
    # print('graph created')
    # print('nx.number_connected_components(G_temp): {}'.format(nx.number_connected_components(G_temp)))
     
    # print('len(G_temp.nodes): {}'.format(nx.number_of_nodes(G_temp)))
    
    # check there is no spliting of graph and number of nodes is same
    if (nx.number_connected_components(G_temp) == 1) and (nx.number_of_nodes(G_temp) == initial_node_count):
    
        omissible_links_index.append(i)
        original_g_df_temp = original_g_df_temp.drop(index = i)
        ctr += 1
        print('[info] Count of admissable links: {}'.format(ctr))
        if ctr == 1000:
            break

In [84]:
len(omissible_links_index)

0

In [ ]:
# create dataframe of removable edges
original_g_df_ghost = original_g_df.loc[omissible_links_index]

# add the target variable 'link'
original_g_df_ghost['link'] = 1

# Reduce the dataset containing non-connected nodes to 2K so that there is a 1:2 ratio of connected nodes to non-connected nodes
# The reduced dataset is a random sample without replacement
data_reduced = data.sample(2000)

data_reduced = data_reduced.append(original_g_df_ghost[['node_1', 'node_2', 'link']], ignore_index=True)

In [ ]:
x = [(model[i] + model[j]) for i,j in zip(data_reduced['node_1'], data_reduced['node_2'])]

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(np.array(x), 
                                                data_reduced['link'], 
                                                test_size = 0.3, 
                                                random_state = 35)

In [ ]:
lr = LogisticRegression(class_weight="balanced")
lr.fit(xtrain, ytrain)

In [ ]:
predictions = lr.predict_proba(xtest)

In [ ]:
roc_auc_score(ytest, predictions[:,1])

In [ ]:
G_karate = nx.karate_club_graph()

In [ ]:
np_g = nx.to_numpy_matrix(G_karate)
print(np_g)
print(np_g.shape)

In [ ]:
import matplotlib.pyplot as plt
pos = nx.spring_layout(G_karate)

In [ ]:
nx.draw(G_karate, cmap = plt.get_cmap('rainbow'), with_labels=True, pos=pos)

In [ ]:
from node2vec import Node2Vec

In [ ]:
from scipy.special import softmax

In [ ]:
import importlib.util
spec = importlib.util.spec_from_file_location("scipy", "/ihome/rboyce/rdb20/.virtualenvs/bionf2071/lib/python3.7/site-packages/scipy/__init__.py")
foo = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = foo 
spec.loader.exec_module(foo)

In [ ]:
import matplotlib.pyplot as plt
from math import isclose
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd
from scipy.special import softmax
from stellargraph import StellarGraph, datasets
from stellargraph.data import EdgeSplitter
from collections import Counter
import multiprocessing
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
dataset = datasets.Cora()
display(HTML(dataset.description))
graph, _ = dataset.load(largest_connected_component_only=True, str_node_ids=True)

The Cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 1433 unique words.

In [ ]:
# Define an edge splitter on the original graph:
edge_splitter_test = EdgeSplitter(graph)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from graph, and obtain the
# reduced graph graph_test with the sampled links removed:
graph_test, examples_test, labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global"
)

print(graph_test.info())

In [ ]:
# Do the same process to compute a training subset from within the test graph
edge_splitter_train = EdgeSplitter(graph_test, graph)
graph_train, examples, labels = edge_splitter_train.train_test_split(
    p=0.1, method="global"
)
(
    examples_train,
    examples_model_selection,
    labels_train,
    labels_model_selection,
) = train_test_split(examples, labels, train_size=0.75, test_size=0.25)

print(graph_train.info())

## Node2Vec

We use Node2Vec [[1]](#refs), to calculate node embeddings. These embeddings are learned in such a way to ensure that nodes that are close in the graph remain close in the embedding space. Node2Vec first involves running random walks on the graph to obtain our context pairs, and using these to train a Word2Vec model.

These are the set of parameters we can use:

* `p` - Random walk parameter "p"
* `q` - Random walk parameter "q"
* `dimensions` - Dimensionality of node2vec embeddings
* `num_walks` - Number of walks from each node
* `walk_length` - Length of each random walk
* `window_size` - Context window size for Word2Vec
* `num_iter` - number of SGD iterations (epochs)
* `workers` - Number of workers for Word2Vec

In [ ]:
p = 1.0
q = 1.0
dimensions = 128
num_walks = 10
walk_length = 80
window_size = 10
num_iter = 1
workers = multiprocessing.cpu_count()

In [ ]:
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec


def node2vec_embedding(graph, name):
    rw = BiasedRandomWalk(graph)
    walks = rw.run(graph.nodes(), n=num_walks, length=walk_length, p=p, q=q)
    print(f"Number of random walks for '{name}': {len(walks)}")

    model = Word2Vec(
        walks,
        size=dimensions,
        window=window_size,
        min_count=0,
        sg=1,
        workers=workers,
        iter=num_iter,
    )

    def get_embedding(u):
        return model.wv[u]

    return get_embedding

In [ ]:
##takes about 5 mins for ~500 edges
embedding_train = node2vec_embedding(graph_train, "Train Graph")

## Train the link prediction model

There are a few steps involved in using the model to perform link prediction:
1. We calculate link/edge embeddings for the positive and negative edge samples by applying a binary operator on the embeddings of the source and target nodes of each sampled edge.
2. Given the embeddings of the positive and negative examples, we train a logistic regression classifier to predict the probability indicating whether an edge between two nodes should exist or not.
3. To use the embeddings in a traditional machine learning classifier, we use binary operators on the embeddings such as average, Hadamard, L1, L2. Here we use the 'average operator' with node embeddings in the logistic regression classifier.


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler

#0. define average function
def average_operator(u, v):
    return (u + v) / 2.0

# 1. link embeddings
def link_examples_to_features(link_examples, transform_node):
    return [
        average_operator(transform_node(src), transform_node(dst))
        for src, dst in link_examples
    ]


# 2. training classifier
def train_link_prediction_model(
    link_examples, link_labels, get_embedding):
    clf = link_prediction_classifier()
    link_features = link_examples_to_features(link_examples, get_embedding)
    clf.fit(link_features, link_labels)
    return clf


def link_prediction_classifier(max_iter=2000):
    lr_clf = LogisticRegressionCV(Cs=10, cv=10, scoring="roc_auc", max_iter=max_iter)
    return Pipeline(steps=[("sc", StandardScaler()), ("clf", lr_clf)])


# 3. return predicted probabilities
def evaluate_link_prediction_model(clf, link_examples_test, link_labels_test, get_embedding):
    link_features_test = link_examples_to_features(link_examples_test, get_embedding)
    predicted = clf.predict_proba(link_features_test)
    return predicted


In [ ]:
def run_link_prediction():
    clf = train_link_prediction_model(
        examples_train, labels_train, embedding_train, average_operator
    )
    predicted = evaluate_link_prediction_model(clf,
                                           examples_model_selection,
                                           labels_model_selection,
                                           embedding_train)

    return predicted

In [ ]:
#Train model - time taken: <1 min
clf = train_link_prediction_model(examples_train, labels_train, embedding_train)

In [ ]:
#Create embeddings of test graph
#Time taken= ~5mins
embedding_test = node2vec_embedding(graph_test, "Test Graph")

In [ ]:
#Get predicted probabilities of test set
predicted = evaluate_link_prediction_model(clf, examples_test, labels_test, embedding_test)

In [ ]:
examples_test

In [ ]:
labels_test

In [ ]:
predicted[:,1]